МФТИ ФИВТ, Машинное обучение, Никита Волков

# Ранжирование

* Дедлайн **14 декабря 23:59** для всех групп.
* В качестве решения задания нужно прислать ноутбук с подробными комментариями.
* Кагл: https://inclass.kaggle.com/c/ml-mipt-ranking4
* Инвайт: https://kaggle.com/join/4uthnui9sndi

### Формат файлов

Вам выдается 4 файла:

* `train.txt` --- обучающая выборка пар запрос-документ и асессорские метки релевантности
* `test.txt` --- тестовая выборка пар запрос-документ
* `queries_test.txt` --- запросы из `train.txt`
* `queries_train.txt` --- запросы из `test.txt`

Колонки в файлах могут быть следующего типа:

* `QueryId` --- уникальный номер запроса
* `DocumentId` --- номер документа, не повторяется для одного запроса
* `DocumentLink` --- url документа
* `Relevance` --- асессорская метка релевантности

Формат файла ответов приведен ниже. Пары запрос-документ должны соответсвовать файлу `test.txt` и должны быть упорядочены по убыванию построенной функции релевантности. То есть так, как в поисковой выдаче.

Качество оценивается с помощью метрики ${NDCG}_{10}$.

In [54]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd
import numpy as np
from collections import defaultdict
import re
import pickle


In [55]:
def get_df_with_bad_urls(filename):
    """Construct dataframe from csv file even if it contains commas in urls.
       We need this function because pandas either throws an error or skips lines when encounters 
       'more columns' than specified in header."""
    with open(filename) as bad_urls_file:
        column_names = bad_urls_file.readline().strip().split(',')
        num_columns = len(column_names)
        lines = bad_urls_file.readlines()
        split_lines = []
        for line in lines:
            tokens = line[:-1].split(',')
            split_line = tokens[:num_columns - 1]
            split_line.append(''.join(tokens[num_columns - 1:]))
            split_lines.append(np.array(split_line))
        split_lines = np.vstack(split_lines)
        df = pd.DataFrame(split_lines, columns=column_names)
        return df

In [78]:
train_queries = pd.read_table('queries_train.txt', header=None)
test_queries = pd.read_table('queries_test.txt', header=None)
train_data = get_df_with_bad_urls('train.txt')
numeric_cols = ['QueryId','DocumentId', 'Relevance']
train_data[numeric_cols] = train_data[numeric_cols].apply(pd.to_numeric)
test_data = get_df_with_bad_urls('test.txt')
test_data[numeric_cols[:-1]] = test_data[numeric_cols[:-1]].apply(pd.to_numeric)

In [57]:
train_data.head()

,Relevance,QueryId,DocumentId,DocumentLink
0,4,1,0,http://otvet.mail.ru/question/195527651/
1,4,1,1,http://otvet.mail.ru/question/195596044/
2,3,1,2,http://ask.fm/Liza_1911
3,3,1,3,https://vk.com/polly_and_viole
4,3,1,4,http://forum.wmj.ru/topic/7969-%D0%BA%D0%B0%D0...


Загружаем документы из test и считаем для них tf-idf фичи:


In [58]:
def visible(element):
    if element.parent.name in ['style', 'script', '[document]', 'head', 'title']:
        return False
    elif re.match('<!--.*-->', str(element)):
        return False
    return True

In [59]:
from multiprocessing.pool import Pool
import urllib

def get_word_freq_url(url):
    word_freqs = defaultdict(int)
    try:
        resource = urllib.request.urlopen(url)
        content = resource.read()
        try:
            content = content.decode(resource.headers.get_content_charset())
        except:
            pass
        soup = BeautifulSoup(content, 'html.parser')
        text = soup.findAll(text=True)
        text = filter(visible, text)
        text = map(lambda s: s.strip(), text)
        text = filter(lambda s: len(s) > 0, text)
        text = " ".join(text).lower()
    except Exception as e:
        print(e)
        text = ""
    tokens = re.split(r'[^\w]+', text)
    for token in tokens:
        word_freqs[token] += 1
    return dict(word_freqs)

def get_word_freqs(data):
    pool = Pool(8)
    data_freqs = pool.map(get_word_freq_url, data["DocumentLink"])
    pool.close()
    return data_freqs

In [60]:
test_freqs = get_word_freqs(test_data)
#test_data[["DocumentLink"]]

<urlopen error [Errno 8] nodename nor servname provided, or not known>


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


IncompleteRead(32288 bytes read)
HTTP Error 404: Not Found
HTTP Error 404: Not Found
<urlopen error [Errno 8] nodename nor servname provided, or not known>


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


HTTP Error 503: Service Unavailable


...

Remote end closed connection without response
HTTP Error 404: Not Found


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Remote end closed connection without response


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


<urlopen error [Errno 60] Operation timed out>


Сериализуем данные, чтобы не потерялись и чтобы не ждать еще раз краулинга

In [61]:
with open('word_freqs_dump', 'wb') as word_freqs_dump:
    pickle.dump(test_freqs, word_freqs_dump)

In [71]:
tfidfs = []
words_in_docs = defaultdict(set)
for ind, freqs in enumerate(test_freqs):
    for word in freqs.keys():
        words_in_docs[word].add(ind)
for ind, freqs in enumerate(test_freqs):
    tfidf = {}
    total_len = sum(freqs.values())
    for word in freqs.keys():
        tfidf[word] = freqs[word] / total_len * np.log(1 + len(test_freqs) / len(words_in_docs[word]))
    tfidfs.append(tfidf)

In [79]:
test_data["TF-IDF"] = tfidfs

In [80]:
test_data.head()

,QueryId,DocumentId,DocumentLink,TF-IDF
0,101,0,https://vk.com/id377383444,"{'таким': 0.0429642819237, 'авг': 0.4296428192..."
1,101,1,http://xn----btbgrjageubg1a.xn--p1ai/420-skuchat,"{'проблемы': 0.00246639233371, '': 0.000493278..."
2,101,2,http://jiznenno.ru/vykhod-iz-treugolnika-karpm...,"{'': 0.000888474504297, 'проблемы': 0.00088847..."
3,101,3,http://iknigi.net/avtor-aleksandra-marinina/11...,"{'': 0.00117563128217, 'занял': 0.001368867281..."
4,101,4,http://fb.ru/article/270014/esli-tufli-veliki-...,"{'проблемы': 0.00789918546894, 'ничего': 0.003..."


In [81]:
test_queries.columns = ["QueryId", "Query"]
test_queries.head()

,QueryId,Query
0,101,Почему нельзя неподошедшего тебе мужика просто...
1,102,"Ты никогда не замечал, какие все вокруг оптими..."
2,103,В какой момент нужно сказать привет когда навс...
3,104,"Кто знает, какие успокоительные лекарства прин..."
4,105,Таджики на уроке физкультуры по команде строим...


In [82]:
test_data = test_data.merge(test_queries, on="QueryId", how="left")
test_data

,QueryId,DocumentId,DocumentLink,TF-IDF,Query
0,101,0,https://vk.com/id377383444,"{'таким': 0.0429642819237, 'авг': 0.4296428192...",Почему нельзя неподошедшего тебе мужика просто...
1,101,1,http://xn----btbgrjageubg1a.xn--p1ai/420-skuchat,"{'проблемы': 0.00246639233371, '': 0.000493278...",Почему нельзя неподошедшего тебе мужика просто...
2,101,2,http://jiznenno.ru/vykhod-iz-treugolnika-karpm...,"{'': 0.000888474504297, 'проблемы': 0.00088847...",Почему нельзя неподошедшего тебе мужика просто...
3,101,3,http://iknigi.net/avtor-aleksandra-marinina/11...,"{'': 0.00117563128217, 'занял': 0.001368867281...",Почему нельзя неподошедшего тебе мужика просто...
4,101,4,http://fb.ru/article/270014/esli-tufli-veliki-...,"{'проблемы': 0.00789918546894, 'ничего': 0.003...",Почему нельзя неподошедшего тебе мужика просто...
5,101,5,https://new.vk.com/wall-56106344_4734187,"{'войти': 0.215443913355, 'поделиться': 0.2154...",Почему нельзя неподошедшего тебе мужика просто...
6,101,6,https://prodoctorov.ru/krasnodar/lpu/8343-uro-...,"{'клинике': 0.038473890059, 'проблемы': 0.0039...",Почему нельзя неподошедшего тебе мужика просто...
7,101,7,http://garri-potter-life.narod.ru/online/hp_ka...,"{'': 0.00400219052299, 'np': 0.00243473231941,...",Почему нельзя неподошедшего тебе мужика просто...
8,101,8,http://otvet.mail.ru/mail/lady.dobrota777,"{'': 0.0174259140172, 'display': 0.01098380653...",Почему нельзя неподошедшего тебе мужика просто...
9,101,9,http://24advokat.ru/cat/v_kakih_sluchayah_mojn...,"{'': 0.00112492872569, 'проблемы': 0.002897770...",Почему нельзя неподошедшего тебе мужика просто...


In [83]:
def get_relevance(dataframe_row):
    tf_idf, query = dataframe_row
    relevance = 0.0
    for word in re.split(r"[^\w]+", query):
        word = word.lower()
        if word in tf_idf:
            relevance += tf_idf[word]
    return relevance

In [84]:
test_data["Relevance"] = test_data[["TF-IDF", "Query"]].apply(get_relevance, axis=1)

In [85]:
test_data

,QueryId,DocumentId,DocumentLink,TF-IDF,Query,Relevance
0,101,0,https://vk.com/id377383444,"{'таким': 0.0429642819237, 'авг': 0.4296428192...",Почему нельзя неподошедшего тебе мужика просто...,0.429643
1,101,1,http://xn----btbgrjageubg1a.xn--p1ai/420-skuchat,"{'проблемы': 0.00246639233371, '': 0.000493278...",Почему нельзя неподошедшего тебе мужика просто...,0.060255
2,101,2,http://jiznenno.ru/vykhod-iz-treugolnika-karpm...,"{'': 0.000888474504297, 'проблемы': 0.00088847...",Почему нельзя неподошедшего тебе мужика просто...,0.011981
3,101,3,http://iknigi.net/avtor-aleksandra-marinina/11...,"{'': 0.00117563128217, 'занял': 0.001368867281...",Почему нельзя неподошедшего тебе мужика просто...,0.032058
4,101,4,http://fb.ru/article/270014/esli-tufli-veliki-...,"{'проблемы': 0.00789918546894, 'ничего': 0.003...",Почему нельзя неподошедшего тебе мужика просто...,0.034441
5,101,5,https://new.vk.com/wall-56106344_4734187,"{'войти': 0.215443913355, 'поделиться': 0.2154...",Почему нельзя неподошедшего тебе мужика просто...,2.080698
6,101,6,https://prodoctorov.ru/krasnodar/lpu/8343-uro-...,"{'клинике': 0.038473890059, 'проблемы': 0.0039...",Почему нельзя неподошедшего тебе мужика просто...,0.003607
7,101,7,http://garri-potter-life.narod.ru/online/hp_ka...,"{'': 0.00400219052299, 'np': 0.00243473231941,...",Почему нельзя неподошедшего тебе мужика просто...,0.004002
8,101,8,http://otvet.mail.ru/mail/lady.dobrota777,"{'': 0.0174259140172, 'display': 0.01098380653...",Почему нельзя неподошедшего тебе мужика просто...,0.050005
9,101,9,http://24advokat.ru/cat/v_kakih_sluchayah_mojn...,"{'': 0.00112492872569, 'проблемы': 0.002897770...",Почему нельзя неподошедшего тебе мужика просто...,0.139587


In [86]:
test_data.sort(["QueryId", "Relevance"], ascending=[1, 0])[["QueryId", "DocumentId"]].to_csv("kaggle.csv", index=False)

/Users/nastya/.virtualenvs/ml/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':
